In [1]:
# Install necessary libraries
!pip install pycurl tqdm pytube libtorrent

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 27.6 MB/s eta 0:00:00


In [2]:
import os
import requests
import subprocess
from getpass import getpass
from tqdm import tqdm
import zipfile
from pytube import YouTube
from getpass import getpass
import sys
import requests
import re
import libtorrent as lt
import time
import warnings
import math

In [3]:
# Create a folder named 'downloads' if it doesn't exist
os.makedirs('downloads', exist_ok=True)

In [4]:
# User inputs
# username = input("Enter your UoM UserName: ")
# password = getpass("Enter your UoM Password: ")

username =  ''
password = ''

if (
    username == '' or
    password == ''
):
    print('Please enter your UoM username and password')

In [5]:
# Function to download a file to the Colab session with progress bar
def download_file(url, local_filename):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        total_size = int(r.headers.get('content-length', 0))
        block_size = 1024  # 1 Kibibyte
        t = tqdm(total=total_size, unit='iB', unit_scale=True, desc=local_filename, ascii=True, unit_divisor=1024)
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=block_size):
                if chunk:  # filter out keep-alive new chunks
                    t.update(len(chunk))
                    f.write(chunk)
        t.close()
        if total_size != 0 and t.n != total_size:
            print("ERROR, something went wrong")
    return local_filename

def download_normal(url):
    # Get file size from the URL header
    response = requests.head(url)
    file_size = int(response.headers.get('content-length', 0))

    file_name = (lambda url: (f"{url.split('/')[-1][:15]}...{url.split('/')[-1][-15:]}" if len(url.split('/')[-1]) > 30 else url.split('/')[-1]))(url)
    local_path = os.path.join('downloads', file_name)
    os.makedirs(os.path.dirname(local_path), exist_ok=True)
    print(f"File Name: {file_name}")
    print(f"File Size: {file_size / (1024 * 1024):.2f} MB")

    # Confirm download
    confirm = input("Do you want to download this file? (y/n): ").strip().lower()
    if confirm == 'y':
        # Download the file
        downloaded_file = download_file(url, local_path)
        print(f'File {file_name} downloaded successfully')
    else:
        print("Download canceled.")

In [6]:
def download_youtube_video(url):
    try:
        yt = YouTube(url)
    except Exception as e:
        print(f"Error: {e}")
        return

    print(f"Title: {yt.title}")
    print(f"Author: {yt.author}")
    print(f"Length: {yt.length} seconds")

    # List available resolutions
    video_streams = yt.streams.filter(progressive=True, file_extension='mp4')
    resolutions = []
    for stream in video_streams:
        resolutions.append(stream.resolution)
    resolutions = list(set(resolutions))  # Remove duplicates

    print("Available Resolutions:")
    for i, res in enumerate(resolutions):
        print(f"{i + 1}. {res}")

    # User selects resolution
    res_choice = int(input("Enter the number of the resolution you want to download: ")) - 1
    selected_resolution = resolutions[res_choice]
    stream = yt.streams.filter(res=selected_resolution, progressive=True).first()

    # Download the video stream with a progress bar
    local_path = os.path.join('downloads', stream.default_filename)
    response = requests.get(stream.url, stream=True)
    total_size = int(response.headers.get('content-length', 0))

    with open(local_path, 'wb') as f, tqdm(
            desc=f'Downloading {stream.default_filename}', total=total_size,
            unit='B', unit_scale=True, unit_divisor=1024,
            ascii=False, miniters=1) as progress_bar:
        for data in response.iter_content(chunk_size=1024):
            f.write(data)
            progress_bar.update(len(data))

    print(f'Video file {stream.default_filename} downloaded successfully')

    # Prompt the user to download subtitles
    download_subtitles = input("Do you want to download subtitles? (y/n): ").strip().lower() == 'y'

    # Attempt to download subtitles if requested
    if download_subtitles:
        try:
            response = requests.get(f'https://www.youtube.com/watch?v={yt.video_id}')
            subtitles = re.findall(r'"captionTracks":\[{\"baseUrl\":\"(.*?)\"', response.text)
            if subtitles:
                subtitle_url = subtitles[0]
                subtitle_file = os.path.join('downloads', f'{yt.title}.srt')
                response = requests.get(subtitle_url)
                with open(subtitle_file, 'wb') as f:
                    f.write(response.content)
                print(f'Subtitles downloaded to {subtitle_file}')
            else:
                print("No subtitles available.")
        except Exception as e:
            print(f"Error while downloading subtitles: {e}")

    return local_path

In [7]:
def download_torrent(url):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)

        ses = lt.session()
        params = {
            'save_path': './downloads',  # Download into the 'downloads' folder
            'storage_mode': lt.storage_mode_t(2),
        }
        handle = lt.add_magnet_uri(ses, url, params)
        sys.stdout.write('Downloading metadata...\n')
        sys.stdout.flush()

        while not handle.has_metadata():
            time.sleep(1)

        sys.stdout.write('Metadata downloaded, starting torrent download...\n')
        sys.stdout.write('Press Ctrl+C to stop\n')
        sys.stdout.flush()

        # Retrieve and print the files and their sizes
        torrent_info = handle.get_torrent_info()
        files = torrent_info.files()
        num_files = files.num_files()
        sys.stdout.write('\nFiles in the torrent:\n')
        for i in range(num_files):
            file_path = files.file_path(i)
            file_size = files.file_size(i)
            sys.stdout.write(f'{i + 1}: {file_path} ({file_size} bytes)\n')
        sys.stdout.flush()

        # Prompt the user to select files to download
        sys.stdout.write('\nEnter the index of the files to download (comma separated), or 0 to download all: ')
        sys.stdout.flush()
        selection = input().strip()

        if selection == '0':
            file_indices = range(num_files)
        else:
            file_indices = [int(x) - 1 for x in selection.split(',') if x.strip().isdigit() and 0 < int(x) <= num_files]

        # Set file priorities
        file_priorities = [0] * num_files
        for index in file_indices:
            file_priorities[index] = 1
        handle.prioritize_files(file_priorities)

        downloaded_Torrent_Files = []

        while True:
            s = handle.status()
            state_str = ['queued', 'checking', 'downloading metadata', 'downloading', 'finished', 'seeding', 'allocating', 'checking fastresume']
            progress = '%.2f%% complete (down: %.1f kB/s up: %.1f kB/s peers: %d) %s' % (
                s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, s.num_peers, state_str[s.state])
            sys.stdout.write('\r' + progress)
            sys.stdout.flush()

            if s.progress >= 1 and s.state in [lt.torrent_status.finished, lt.torrent_status.seeding]:
                break

            time.sleep(1)
            downloaded_Torrent_Files = [file.path for file in handle.get_torrent_info().files()]

        sys.stdout.write('\nDownload complete!\n')
        sys.stdout.flush()

In [8]:
def upload_file(username, password, file_path, upload_point):
    # Remove special characters from the filename
    file_name = re.sub(r'[^\w\s.-]', '', os.path.basename(file_path))

    # Rename the file if the filename has changed
    if file_name != os.path.basename(file_path):
        new_file_path = os.path.join(os.path.dirname(file_path), file_name)
        os.rename(file_path, new_file_path)
        file_path = new_file_path

    login_detail = f'"{username}:{password}"'
    header = '"OCS-APIRequest: true"'
    curl_command = f'curl -u {login_detail} --progress-bar -T "{file_path}" {upload_point}'
    print(f"\nUploading - {file_path}")

    # Start the curl command in a subprocess with stdout redirected
    process = subprocess.Popen(curl_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

    try:
        # Initialize variables to store the previous progress
        prev_progress = 0

        # Loop through the output of the command
        for line in process.stderr:
            # Extract progress information using regex
            match = re.search(r'(\d+(\.\d+)?)%', line)
            if match:
                progress = float(match.group(1))

                # Calculate the number of characters to represent the progress bar
                bar_length = 50
                filled_length = int(bar_length * progress / 100)
                filled_bar = '█' * filled_length  # Use '█' for the filled part
                empty_bar = '░' * (bar_length - filled_length)  # Use '░' for the empty part

                # Display progress information
                sys.stdout.write(f"\rUpload Progress: [{filled_bar}{empty_bar}] {progress:.1f}%")
                sys.stdout.flush()

                # Update previous progress
                prev_progress = progress
            else:
                # Skip lines that don't contain progress updates
                continue

        # Wait for the process to finish and get the return code
        return_code = process.wait()

        # Check if the command was successful
        if return_code == 0:
            print(f"\nFile {file_path} uploaded to DMS successfully.")
        else:
            print("\nError occurred during file upload.")
    except Exception as e:
        print("\nAn error occurred:", e)

In [9]:
def zip_files(file_list, zip_name):
    # Calculate the total size of the files
    total_size = sum(os.path.getsize(file) for file in file_list)
    approximate_zip_size = total_size * 0.95  # assuming an average compression ratio of 96%
    print(f"Approximate size of the zip file: {approximate_zip_size / (1024 * 1024):.2f} MB")

    # Create the zip file
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        for file in file_list:
            arcname = os.path.relpath(file, 'downloads')
            zipf.write(file, arcname=arcname)

    # Get the actual size of the created zip file
    actual_zip_size = os.path.getsize(zip_name)
    print(f"Actual size of the zip file: {actual_zip_size / (1024 * 1024):.2f} MB")

    # Ask if the user wants to split the zip file
    parts = int(input("Enter the number of parts to split the zip file (or 0 for no split): ").strip())

    if parts > 1:
        part_size = math.ceil(actual_zip_size / parts)
        print(f"Each part will be approximately {part_size / (1024 * 1024):.2f} MB")

        # Split the zip file into parts
        part_num = 1
        part_names = []
        with open(zip_name, 'rb') as f:
            part = f.read(part_size)
            while part:
                part_name = f"{zip_name}.part{part_num:03d}"
                part_names.append(part_name)
                with open(part_name, 'wb') as part_file:
                    part_file.write(part)
                part = f.read(part_size)
                part_num += 1

        os.remove(zip_name)  # remove the original single zip file

        print("Files zipped successfully as", part_names)
        return part_names
    else:
        print("Files zipped successfully as", [zip_name])
        return [zip_name]

In [10]:
def get_files_and_folders():
    files = []
    folders = []

    # Iterate through all items in the 'downloads' folder
    for item in os.listdir('downloads'):
        item_path = os.path.join('downloads', item)
        if os.path.isfile(item_path):
            files.append(item_path)
        elif os.path.isdir(item_path):
            folders.append(item_path)

    return files, folders

def display_items(items):
    for idx, item in enumerate(items, start=1):
        print(f"{idx}. {os.path.basename(item)}")

def select_files(files, folders, selected_indices):
    selected_files = []
    if selected_indices == '0':
        selected_files = files + [os.path.join(folder, file) for folder in folders for file in os.listdir(folder) if os.path.isfile(os.path.join(folder, file))]

    else:
        try:
            for index in selected_indices.split(','):
                index = int(index.strip())
                if 1 <= index <= len(folders):
                    folder_files = [os.path.join(folders[index - 1], file) for file in os.listdir(folders[index - 1]) if os.path.isfile(os.path.join(folders[index - 1], file))]
                    selected_files.extend(folder_files)
                    display_items(folder_files)
                elif len(folders) < index <= len(folders) + len(files):
                    selected_files.append(files[index - len(folders) - 1])
                else:
                    print(f"Invalid index: {index}")
        except ValueError:
            print("Invalid input. Please enter valid indices.")
    return selected_files

def handle_upload():
    files, folders = get_files_and_folders()

    print("Folders and Files:")
    display_items(folders + files)

    selected_indices = input("Enter the indices of the items you want to upload (comma-separated, or 0 for all): ").strip()

    selected_files = select_files(files, folders, selected_indices)

    return selected_files

def handle_zip():
    files, folders = get_files_and_folders()

    print("Folders and Files:")
    display_items(folders + files)

    selected_indices = input("Enter the indices of the items you want to zip (comma-separated, or 0 for all): ").strip()

    selected_files = select_files(files, folders, selected_indices)

    zip_name = input("Enter the name for the zip file: ").strip()

    zip_file = zip_files(selected_files, os.path.join('downloads', f'{zip_name}.zip'))

    print(f"Files zipped successfully as {zip_file}")

def upload_files(files):
    upload_point = "https://dms.uom.lk/remote.php/webdav/ColabDownloads/"
    files_to_upload = files

    for file in files_to_upload:
        upload_file(username, password, file, upload_point)


In [11]:
# #Join splitted zip files
# import os
# import zipfile
# import math

# def join_zip_parts(output_zip_name):
#     base_name = input("Enter the base name of the zip file (without the .part### extension): ")
#     part_files = [file for file in os.listdir() if file.startswith(base_name) and file.split('.')[-1].startswith('part')]
#     part_files.sort()  # Ensure parts are in correct order

#     print('No. of parts found:', len(part_files))
#     print('\n'.join(part_files))

#     if input ('Confirm (y/n)? ') != 'y':
#         print('Exited.')
#         return

#     # Check if the output_zip_name already exists
#     if os.path.exists(output_zip_name):
#         # Append a number to the output_zip_name to ensure uniqueness
#         count = 1
#         while os.path.exists(f"{output_zip_name}_{count}.zip"):
#             count += 1
#         output_zip_name = f"{output_zip_name}_{count}.zip"

#     # Calculate total size of parts
#     total_size = sum(os.path.getsize(part_file) for part_file in part_files)
#     progress = 0

#     with open(output_zip_name, 'wb') as output_file:
#         for part_file in part_files:
#             with open(part_file, 'rb') as pf:
#                 # Read and write part file
#                 part_data = pf.read()
#                 output_file.write(part_data)
#                 # Update progress
#                 progress += len(part_data)
#                 percentage = (progress / total_size) * 100
#                 print(f"Progress: {percentage:.2f}%")

#     input(f"Reassembled zip file: {output_zip_name}")

# def split_zip(zip_file, num_parts):
#     # Check if the zip file exists
#     if not os.path.exists(zip_file):
#         print("Error: Zip file does not exist.")
#         return

#     # Open the zip file in read-binary mode
#     with open(zip_file, 'rb') as f:
#         # Read the entire contents of the zip file
#         zip_data = f.read()

#     # Calculate the size of each part
#     zip_size = len(zip_data)
#     part_size = math.ceil(zip_size / num_parts)

#     # Split the zip data into parts
#     zip_parts = [zip_data[i:i+part_size] for i in range(0, zip_size, part_size)]

#     # Write each part to a separate file
#     part_files = []
#     for i, part in enumerate(zip_parts):
#         part_file = f"{zip_file}.part{i+1:03d}"
#         with open(part_file, 'wb') as f:
#             f.write(part)
#         part_files.append(part_file)

#     print(f"{num_parts} parts created successfully.")
#     return part_files

# # Example usage
# #split_zip('Reassembled.zip', 3)

# # Example usage
# join_zip_parts('Reassembled.zip')

In [12]:
url = input("Enter the URL of the file to download:")
#url = 'https://files2.lmhapks.com/MOD/life-is-strange-MOD-1.00.310__LMHAPKS.COM-installer.apk'

Enter the URL of the file to download:magnet:?xt=urn:btih:4B3DCE31C713B02726F67E2EF49DE9FF965EC3B7&dn=Inside+Out+%282015%29+%5B720p%5D+%5BBluRay%5D+%5BYTS.CONTACT%5D&tr=udp://tracker.opentrackr.org:1337/announce&tr=udp://tracker.leechers-paradise.org:6969/announce&tr=udp://9.rarbg.to:2710/announce&tr=udp://p4p.arenabg.ch:1337/announce&tr=udp://tracker.cyberia.is:6969/announce&tr=http://p4p.arenabg.com:1337/announce&tr=udp://tracker.internetwarriors.net:1337/announce


In [13]:
if url.startswith('magnet:?xt=urn:btih:'):
  download_torrent(url)
elif (('youtube.com' in url) or ('youtu.be' in url)):
  download_youtube_video(url)

Metadata downloaded, starting torrent download...
Press Ctrl+C to stop

Files in the torrent:
1: Inside Out (2015)/Inside.Out.2015.720p.BluRay.x264.YIFY.mp4 (791242383 bytes)
2: Inside Out (2015)/WWW.YTS.TO.jpg (105983 bytes)

Enter the index of the files to download (comma separated), or 0 to download all: 1
100.00% complete (down: 3718.3 kB/s up: 150.8 kB/s peers: 6) seeding
Download complete!


In [ ]:
#download from .torrent
path = input("Enter the path of torrent: ")
torrent_file = open(path, 'rb')
torrent_info = lt.torrent_info(torrent_file.read())
magnet_uri = lt.make_magnet_uri(torrent_info)
download_torrent(magnet_uri)

In [15]:
upload_files(handle_upload())

Folders and Files:
1. Inside Out (2015)
2. .4b3dce31c713b02726f67e2ef49de9ff965ec3b7.parts
3. Inside.Out.2015.720p.BluRay.x264.YIFY.mp4
Enter the indices of the items you want to upload (comma-separated, or 0 for all): 3

Uploading - downloads/Inside.Out.2015.720p.BluRay.x264.YIFY.mp4
Upload Progress: [██████████████████████████████████████████████████] 100.0%
File downloads/Inside.Out.2015.720p.BluRay.x264.YIFY.mp4 uploaded to DMS successfully.


In [ ]:
handle_zip()

In [ ]:
download_normal(url)

File Name: life-is-strange...M-installer.apk
File Size: 999.80 MB
Do you want to download this file? (y/n): Y


downloads/life-is-strange...M-installer.apk: 100%|##########| 0.98G/0.98G [00:59<00:00, 17.5MiB/s]

File life-is-strange...M-installer.apk downloaded successfully
